In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

import gomoku, player_basic, player_zero, player_zero4
from time import time

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

In [ ]:
## Gobal parameters ==========
SIZE = 9
WIN = 5
N_ITER = 500
## ===========================

# Playtest against cp

In [ ]:
## ===human first?===========
human_first = True
## =======================

game = gomoku.Gomoku(SIZE, WIN)
cp_turn = -1 if human_first else +1

**Pick opponent model**

In [ ]:
## Baseline Model
p2 = player_basic.FeaturePlayer('p2', cp_turn)

In [ ]:
## AlphaGo Zero, 1-feature
model2 = player_zero.net(SIZE)
model2.load_weights('1feat 9x9/day26.h5')
p2 = player_zero.ZeroPlayer('p2', cp_turn, game, model2, N_ITER)
p2.tree.temp = .1 # reduce exploration

In [ ]:
## AlphaGo Zero, 4-feature
model2 = player_zero4.net(SIZE)
model2.load_weights('4feat 9x9/day10.h5')
p2 = player_zero4.ZeroPlayer('p2', cp_turn, game, model2, N_ITER)
p2.tree.temp = .1 # reduce exploration

**Play against 1-feature AI**

In [ ]:
if not human_first:
    game.play(*p2.play(game))
game.show()

In [ ]:
## play your move ==========
game.play(4,4)
## =========================

t = time()
if not game.finished:
    game.play(*p2.play(game))
game.show()
print("time: {0:.3f}".format(time() - t))

# Playtest cp against cp

In [ ]:
game = gomoku.Gomoku(SIZE, WIN)

## ===edit players as you see fit======================
## p1 is black
model1 = player_zero4.net(SIZE)
model1.load_weights('4feat 9x9/day10.h5')
p1 = player_zero4.ZeroPlayer('p1', +1, game, model1, N_ITER)
p1.tree.temp = .1 # reduce exploration

## p2 is white
model2 = player_zero4.net(SIZE)
model2.load_weights('4feat 9x9/day9.h5')
p2 = player_zero4.ZeroPlayer('p2', -1, game, model2, N_ITER)
p2.tree.temp = .1 # reduce exploration
## ====================================================

pa, pb = p1, p2

In [ ]:
t = time()
#if not game.finished: # one move at a time
while not game.finished: # play to terminal state
    game.play(*pa.play(game))
    pa, pb = pb, pa
game.show()
print("time: {0:.3f}".format(time() - t))

# Batchtest cp vs cp

In [ ]:
black_wins = 0
white_wins = 0
for i in range(10):
    game = gomoku.Gomoku(SIZE, WIN)
    ## ===edit players as you see fit======================
    ## p1 is black
    #p1 = player_basic.FeaturePlayer('p1', +1)
    model1 = player_zero4.net(SIZE)
    model1.load_weights('4feat 9x9/day10.h5')
    p1 = player_zero4.ZeroPlayer('p1', +1, game, model1, N_ITER)
    p1.tree.temp = .1 # reduce exploration

    ## p2 is white
    #p2 = player_basic.FeaturePlayer('p2', -1)
    model2 = player_zero4.net(SIZE)
    model2.load_weights('4feat 9x9/day9.h5')
    p2 = player_zero4.ZeroPlayer('p2', -1, game, model2, N_ITER)
    p2.tree.temp = .1 # reduce exploration
    ## ====================================================
    pa, pb = p1, p2

    while not game.finished:
        game.play(*pa.play(game))
        pa, pb = pb, pa
        
    if game.winner == +1:
        black_wins += 1
    elif game.winner == -1:
        white_wins += 1
    
    game.show()
    
print("\nblack wins:", wins)
print("white wins:", losses)